**I - Import the librairies and modules**

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

pd.set_option('display.max_columns', None)

**II - Import the data** : data set is df

In [11]:
df = pd.read_csv("../raw_data/FC26_20250921.csv", low_memory=False)

In [12]:
df.head(5)

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,252371,/player/252371/jude-bellingham/260004/,26,4,2025-09-19,J. Bellingham,Jude Victor William Bellingham,"CAM, CM",90,94,174500000,320000,22,2003-06-29,186,75,53.0,La Liga,1.0,243.0,Real Madrid,CAM,5.0,NaN,2023-07-01,2029.0,14,England,1318.0,CAM,10.0,Right,4,4,5,NaN,Normal (170-185),Yes,370800000.0,"#Dribbler, #Playmaker , #Clinical finisher, #C...","Relentless +, Low Driven Shot, Gamechanger, In...",80.0,86.0,83.0,90.0,78.0,85.0,66,88,75,90,77,91,73,68,89,91,81,80,83,91,83,86,85,94,80,87,85,82,91,90,74,90,77,79,77,14,11,10,5,8,NaN,86+3,86+3,86+3,87+0,89+0,89+0,89+0,87+0,89+3,89+3,89+3,87+3,89+3,89+3,89+3,87+3,83+3,85+3,85+3,85+3,83+3,82+3,81+3,81+3,81+3,82+3,18+3,https://cdn.sofifa.net/players/252/371/26_120.png
1,239053,/player/239053/federico-valverde/260004/,26,4,2025-09-19,F. Valverde,Federico Santiago Valverde Dipetta,"CM, CDM, RB",89,90,120500000,340000,26,1998-07-22,182,74,53.0,La Liga,1.0,243.0,Real Madrid,RDM,8.0,NaN,2016-07-22,2029.0,60,Uruguay,NaN,NaN,NaN,Right,4,3,4,NaN,Unique,Yes,256100000.0,"#Playmaker , #Distance shooter, #Tackling , #T...","Relentless +, Power Shot, Long Ball Pass, Inte...",88.0,84.0,84.0,84.0,83.0,85.0,78,80,63,88,78,84,76,69,88,88,84,92,77,89,68,91,82,93,82,91,81,86,86,86,59,84,82,87,86,6,10,6,15,8,NaN,84+3,84+3,84+3,85+0,86+0,86+0,86+0,85+0,86+3,86+3,86+3,86+3,88+2,88+2,88+2,86+3,86+3,87+3,87+3,87+3,86+3,86+3,83+3,83+3,83+3,86+3,18+3,https://cdn.sofifa.net/players/239/053/26_120.png
2,212622,/player/212622/joshua-kimmich/260004/,26,4,2025-09-19,J. Kimmich,Joshua Walter Kimmich,"CDM, RB, CM",89,89,86000000,140000,30,1995-02-08,177,75,19.0,Bundesliga,1.0,21.0,FC Bayern München,RDM,6.0,NaN,2015-07-01,2029.0,21,Germany,1337.0,RCM,6.0,Right,4,3,4,NaN,Normal (170-185),Yes,141900000.0,"#Playmaker , #Crosser, #Tackling , #Complete m...","Long Ball Pass +, Finesse Shot, Whipped Pass, ...",72.0,74.0,89.0,84.0,83.0,79.0,92,68,72,89,69,82,85,79,92,86,75,70,83,89,84,78,77,92,69,83,89,85,78,86,71,86,82,86,85,8,15,7,15,15,NaN,77+3,77+3,77+3,82+0,81+0,81+0,81+0,82+0,83+3,83+3,83+3,84+3,87+2,87+2,87+2,84+3,86+3,87+2,87+2,87+2,86+3,85+3,82+3,82+3,82+3,85+3,21+3,https://cdn.sofifa.net/players/212/622/26_120.png
3,235212,/player/235212/achraf-hakimi/260004/,26,4,2025-09-19,A. Hakimi,Achraf Hakimi Mouhأشرف حكيمي,"RB, RM",89,90,111000000,170000,26,1998-11-04,181,73,16.0,Ligue 1,1.0,73.0,Paris Saint-Germain,RB,2.0,NaN,2021-07-06,2029.0,129,Morocco,111111.0,RB,2.0,Right,4,4,4,NaN,Lean (170-185),Yes,213700000.0,#Speedster,"Relentless +, Low Driven Shot, Whipped Pass, J...",92.0,79.0,82.0,83.0,82.0,79.0,85,80,71,86,74,82,78,80,73,85,89,95,81,90,

**III - Clean the data manually, start the analysis**

In [13]:
# Visualise our feature in a list : Numerical and Categorical features
features_columns = df.columns
features_columns

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk',
       'player_face_url'],
      dtype='object', length=110)

In [29]:
primary_position='club_position'

In [15]:
# Convert player position string into list of positions
player_features_df['player_positions'] = player_features_df['player_positions'].apply(lambda x: x.split(', '))

# Take the first given position as a player's primary position (new column)
player_features_df['primary_position'] = player_features_df['player_positions'].apply(lambda x : x[0])

In [16]:
numeric_columns = ['overall', 'potential', 'height_cm', 'weight_kg', 'weak_foot', 'skill_moves',
                'international_reputation', 'pace', 'shooting', 'passing', 'dribbling', 'defending',
                'physic', 'attacking_crossing', 'attacking_finishing','attacking_heading_accuracy',
                'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve',
                'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
                'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                'movement_reactions', 'movement_balance', 'power_shot_power',
                'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
                'mentality_aggression', 'mentality_interceptions',
                'mentality_positioning', 'mentality_vision', 'mentality_penalties',
                'mentality_composure', 'defending_marking_awareness',
                'defending_standing_tackle', 'defending_sliding_tackle',
                'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']

categorical_columns = ['preferred_foot', 'player_positions', 'primary_position']

**IV - Scale the data : categories**

In [17]:
# Positional groups and longer positional names if needed

# Map primary position to broader groups
position_groups = {
    'ST': 'Forward', 'CF': 'Forward', 'LW': 'Forward', 'RW': 'Forward',
    'CAM': 'Midfielder', 'CM': 'Midfielder', 'CDM': 'Midfielder', 'LM': 'Midfielder', 'RM': 'Midfielder',
    'CB': 'Defender', 'LB': 'Defender', 'RB': 'Defender', 'LWB': 'Defender', 'RWB': 'Defender',
    'GK': 'Goalkeeper'
}
# player_features_df['position_group'] = player_features_df['primary_position'].map(position_groups).fillna('Other')

# Dictionary matching long position names to short
{'CM': 'Central Midfielder',
'CB': 'Central Defender',
'RM': 'Right Midfielder',
'LM': 'Left Midfielder',
'ST': 'Striker',
'CAM': 'Central Attacking Midfielder',
'CDM': 'Central Defensive Midfielder',
'RB': 'Right Fullback',
'LB': 'Left Fullback',
'GK': 'Goalkeeper',
'LW': 'Left Winger',
'RW': 'Right Winger'}

{'CM': 'Central Midfielder',
 'CB': 'Central Defender',
 'RM': 'Right Midfielder',
 'LM': 'Left Midfielder',
 'ST': 'Striker',
 'CAM': 'Central Attacking Midfielder',
 'CDM': 'Central Defensive Midfielder',
 'RB': 'Right Fullback',
 'LB': 'Left Fullback',
 'GK': 'Goalkeeper',
 'LW': 'Left Winger',
 'RW': 'Right Winger'}

**V - Pipeline for preprocessing the data set**

In [18]:
#import sklearn and module
from sklearn.pipeline import Pipeline

**One Hot Encode categorical features in order to scale our categorical data btwn O and 1OHE player primary positions**

In [26]:
categorical_columns = ['preferred_foot', 'player_positions', 'primary_position']
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(player_features_df[['primary_position']])
player_features_df[ohe.get_feature_names_out()] = ohe.transform(player_features_df[['primary_position']])

# OHE player prefered foot
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(player_features_df[['preferred_foot']])
player_features_df[ohe.get_feature_names_out()] = ohe.transform(player_features_df[['preferred_foot']])

# OHE player player position is not used because of redondancy
#ohe = OneHotEncoder(sparse_output=False)
#ohe.fit(player_features_df[['player_position']])
#player_features_df[ohe.get_feature_names_out()] = ohe.transform(player_features_df[['player_position']])

**However a 2nd solution would be to use a Random Forest Classifier - The interest would be to clean the dataset, preprocess in a single compute the data and classify without label**

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
#pipeline = Pipeline ([
    #("Cleaner", type_cleaner),
    #("Preprocessing", preproc),
    #("Classifier", RandomForestClassifier())])

**A 3rd solution would be to use a KNN on our categories. We can rely on the OHE realised yesterday**


In [22]:
# import pipeline ok
# import Simple Imputer : to fill any missing data although we should have a clean dataset this is for security
from sklearn.impute import SimpleImputer
# import Standard Scalers to harmonize our data. I choose to use the 3 of them for later accuracy evaluation
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer

In [40]:
# liste des colonnes catégorielles (remplace par tes noms de colonnes)
cat_cols = ['preferred_foot', 'player_positions']

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

Cat_preproc = ColumnTransformer(
    transformers=[
        ('cat', cat_pipeline, cat_cols)
    ],
    remainder='passthrough'  # ou 'drop' selon ce que tu veux faire des colonnes non listées
)
cat_pipeline

,steps,"[('imputer', ...), ('ohe', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'constant'
,fill_value,'missing'
,copy,True
,add_indicator,False
,keep_empty_features,False
,categories,'auto'


In [37]:
import pandas as pd

Cat_preproc.fit(df[['preferred_foot','player_positions']])
Xt = Cat_preproc.transform(df[['preferred_foot','player_positions']])

print (Xt)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 32313 stored elements and shape (18405, 648)>
  Coords	Values
  (0, 0)	1.0
  (0, 6)	1.0
  (1, 0)	1.0
  (1, 177)	1.0
  (2, 0)	1.0
  (2, 138)	1.0
  (3, 0)	1.0
  (3, 458)	1.0
  (4, 0)	1.0
  (4, 148)	1.0
  (5, 0)	1.0
  (5, 111)	1.0
  (6, 281)	1.0
  (7, 0)	1.0
  (7, 165)	1.0
  (8, 0)	1.0
  (8, 111)	1.0
  (9, 601)	1.0
  (10, 281)	1.0
  (11, 0)	1.0
  (11, 6)	1.0
  (12, 0)	1.0
  (12, 576)	1.0
  (13, 0)	1.0
  (13, 458)	1.0
  :	:
  (18392, 256)	1.0
  (18393, 0)	1.0
  (18393, 256)	1.0
  (18394, 0)	1.0
  (18394, 256)	1.0
  (18395, 0)	1.0
  (18395, 256)	1.0
  (18396, 0)	1.0
  (18396, 256)	1.0
  (18397, 0)	1.0
  (18397, 256)	1.0
  (18398, 0)	1.0
  (18398, 256)	1.0
  (18399, 0)	1.0
  (18399, 256)	1.0
  (18400, 0)	1.0
  (18400, 256)	1.0
  (18401, 0)	1.0
  (18401, 256)	1.0
  (18402, 0)	1.0
  (18402, 256)	1.0
  (18403, 0)	1.0
  (18403, 256)	1.0
  (18404, 0)	1.0
  (18404, 256)	1.0


In [39]:
Cat_preproc.get_feature_names_out

<bound method ColumnTransformer.get_feature_names_out of ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ohe',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['preferred_foot', 'player_positions'])])>

**Models**
Logistic Regression would be interesting to make since it is a classifier. It uses a Sigmoid Function to map the probability of belonging to a class. Since you encoded the data 'overall' and 'potential', this would make sense to define à thresold

import matplotlib.pyplot as plt

# Plot preferred_foot vs player_positions
plt.scatter(df['preferred_foot'], df['player_positions'])

In [56]:
from sklearn.neighbors import KNeighborsRegressor

# créer le modèle
knn = KNeighborsRegressor(
    n_neighbors=5,       # nombre de voisins
    weights='uniform',   # pondération uniforme (chaque voisin compte pareil)
    algorithm='auto',    # choix automatique de l'algorithme
    leaf_size=30,
    p=2,                 # distance euclidienne (p=2)
    metric='minkowski'   # métrique de distance
)
y = df['overall'] 

In [57]:
knn.fit(Xt, y)  # Xt = features transformées par Cat_preproc

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [58]:
y_pred = knn.predict(Xt)
print(y_pred)

[68.2 57.6 63.4 ... 66.  66.  66. ]


In [60]:
print(y_pred.mean())

64.48357511545775
